Design of a new game, Spuds

In [1]:
using Random
using DataFrames
using CSV
using Statistics
using Printf
using LinearAlgebra

In [2]:
struct Spud
    name::String
    h::Int64
    f::Int64
    l::Int64
    p::Int64
    r::Int64
    s::Int64
end

const MXS = 100
const MXV = 20
const MNV = 1

function cost(h::Int64, f::Int64, l::Int64, p::Int64, r::Int64, s::Int64)::Int64
    #return h*(f+l-2*MNV) + f*(p+r+s-3*MNV) + h+f+l+p+r+s-6*MNV
    return h*(f+l) + f*(p+r+s)
end

cost (generic function with 1 method)

In [3]:
function eval_battle(a::Spud, b::Spud)::Int
    a_finds = a.f >= b.h
    b_finds = b.f >= a.h
    #score_a = 4 * (1000*a.p + a.l > 1000*b.p+b.l) + 3 * (1000*a.r+a.l > 1000*b.r+b.l) + 2 * (1000*a.s+a.l > 1000*b.s+b.l)
    #score_b = 4 * (1000*a.p + a.l < 1000*b.p+b.l) + 3 * (1000*a.r+a.l < 1000*b.r+b.l) + 2 * (1000*a.s+a.l < 1000*b.s+b.l)
    score_a = 4 * (a.p > b.p) + 3 * (a.r > b.r) + 2 * (a.s > b.s)
    score_b = 4 * (a.p < b.p) + 3 * (a.r < b.r) + 2 * (a.s < b.s)
    melee_win = sign(score_a - score_b)
    if a_finds && b_finds
        tiebreaker = 4 * melee_win + 2 * sign(a.l - b.l) + sign(a.h - b.h)
        return sign(tiebreaker)
    end
    if a_finds && !b_finds
        return 1
    end
    if !a_finds && b_finds
        return -1
    end
    if !a_finds && !b_finds
        tiebreaker = 4 * sign(a.l - b.l) + 2 * melee_win + sign(a.f - b.f)
        return sign(tiebreaker)
    end
end


eval_battle (generic function with 1 method)

In [4]:
tab = CSV.read("census_yob2022_names.txt", DataFrame, header = false)
names = tab.Column1
adjectives = CSV.read("adjectives.csv", DataFrame)
nouns = CSV.read("nouns.csv", DataFrame)
jobs = CSV.read("jobs.csv", DataFrame)

function random_name_and_stat()::Spud
    vp = [0, 0, 0, 0, 0, 0]
    #nametype = rand([1,1,1,1,1,2,2,2,3])
    name = ""
    noun = ""
    adj = ""
    noun_i = rand(1:nrow(nouns))
    noun = nouns[noun_i, :noun]
    vp[1] = vp[1] + nouns[noun_i, :H]
    vp[2] = vp[2] + nouns[noun_i, :F]
    vp[3] = vp[3] + nouns[noun_i, :L]
    vp[4] = vp[4] + nouns[noun_i, :P]    
    vp[5] = vp[5] + nouns[noun_i, :R]
    vp[6] = vp[6] + nouns[noun_i, :S]    
    adj_i = rand(1:nrow(adjectives))
    adj = adjectives[adj_i, :adjective]
    vp[1] = vp[1] + adjectives[adj_i, :H]
    vp[2] = vp[2] + adjectives[adj_i, :F]
    vp[3] = vp[3] + adjectives[adj_i, :L]
    vp[4] = vp[4] + adjectives[adj_i, :P]
    vp[5] = vp[5] + adjectives[adj_i, :R]
    vp[6] = vp[6] + adjectives[adj_i, :S]
    job_i = rand(1:nrow(jobs))
    job = jobs[job_i, :job]
    vp[1] = vp[1] + jobs[job_i, :H]
    vp[2] = vp[2] + jobs[job_i, :F]
    vp[3] = vp[3] + jobs[job_i, :L]
    vp[4] = vp[4] + jobs[job_i, :P]
    vp[5] = vp[5] + jobs[job_i, :R]
    vp[6] = vp[6] + jobs[job_i, :S]
    name = string(adj, " ", noun, " the ", job)
    Spud(name, vp[1], vp[2], vp[3], vp[4], vp[5], vp[6])
end

random_name_and_stat (generic function with 1 method)

In [5]:
function rand_rename(a::Spud, n_tries::Int = 100)::Spud
    best_score = 0.0
    best_b = random_name_and_stat()
    for ii in 1:n_tries
        b = random_name_and_stat()
        b_norm = sqrt(b.h^2 + b.f^2 + b.l^2 + b.p^2 + b.r^2 + b.s^2)
        score = (a.h * b.h + a.f * b.f + a.l * b.l + a.p * b.p + a.r * b.r + a.s * b.s)/b_norm
        if score > best_score
            best_score = score
            best_b = b
        end
    end
    return Spud(best_b.name, a.h, a.f, a.l, a.p, a.r, a.s)
end

rand_rename (generic function with 2 methods)

In [6]:
function eval_battle_list(a::Spud, bs::Array{Spud})::Int
    score = 0
    for ii in 1:length(bs)
        score = score + eval_battle(a, bs[ii])
    end
    return score
end

function eval_battle_list2(a::Spud, bs::Array{Spud}, w::Vector{Float64})::AbstractFloat
    score = 0.0
    for ii in 1:length(bs)
        score = score + w[ii] * eval_battle(a, bs[ii])
    end
    return score
end


function pick_best(as::Array{Spud}, bs::Array{Spud})::Spud
    bestscore = -999
    bestf = as[1]
    for ii in 1:length(as)
        score = eval_battle_list(as[ii], bs)
        if score > bestscore
            bestscore = score
            bestf = as[ii]
        end
    end
    return bestf
end

function pick_best_rdmly(as::Array{Spud}, bs::Array{Spud}, ntries::Int)::Spud
    bestscore = -999
    bestf = rand(as)
    for ii in 1:ntries
        f = rand(as)
        score = eval_battle_list(f, bs)
        if score > bestscore
            bestscore = score
            bestf = f
        end
    end
    return bestf
end

function pick_best_rdmly_g(g::Function, bs::Array{Spud}, ntries::Int)::Spud
    bestscore = -999
    bestf = g()
    for ii in 1:ntries
        f = g()
        score = eval_battle_list(f, bs)
        if score > bestscore
            bestscore = score
            bestf = f
        end
    end
    return bestf
end

function spuds_to_df(as::Array{Spud})::DataFrame
    names = Array{String}(undef, length(as))
    hs = Array{Int}(undef, length(as))
    fs = Array{Int}(undef, length(as))
    ls = Array{Int}(undef, length(as))
    ps = Array{Int}(undef, length(as))
    rs = Array{Int}(undef, length(as))
    ss = Array{Int}(undef, length(as))
    for ii in 1:length(as)
        names[ii] = as[ii].name
        hs[ii] = as[ii].h
        fs[ii] = as[ii].f
        ls[ii] = as[ii].l
        ps[ii] = as[ii].p
        rs[ii] = as[ii].r
        ss[ii] = as[ii].s
    end
    df = DataFrame(name = names, h = hs, f = fs, l = ls, p = ps, r = rs, s = ss)
    return df
end

function fpart(x::AbstractFloat)::AbstractFloat
  return x - trunc(x)
end

function eval_team_battle(as::Array{Spud}, bs::Array{Spud})::Int
    a_i = 1
    b_i = 1
    while (a_i <= length(as)) && (b_i <= length(bs))
        res = eval_battle(as[a_i], bs[b_i])
        if res == 1
            b_i = b_i + 1
        else
            a_i = a_i + 1
            if res == 0
                b_i = b_i + 1
            end
        end
    end
    a_out = (a_i > length(as))
    b_out = (b_i > length(as))
    if a_out
        if b_out
            return 0
        else
            return -1
        end
    else
        return 1
    end
end

function compare_generator(f1, f2, limit)
    a_i = 1
    b_i = 1
    f_a = f1()
    f_b = f2()
    while (a_i < limit) && (b_i < limit)
        res = eval_battle(f_a, f_b)
        if res != -1
            b_i = b_i + 1
            f_b = f2()
        end
        if res != 1
            a_i = a_i + 1
            f_a = f1()
        end
    end
    return (a_i/limit, b_i/limit)
end

function random_team(f::Function, team_size::Int)::Array{Spud}
    team = Array{Spud}(undef, team_size)
    for i in 1:team_size
        team[i] = f()
    end
    return team
end

random_team (generic function with 1 method)

In [7]:
function upgrade_spud(sp::Spud)::Spud
    h = sp.h
    f = sp.f
    l = sp.l
    p = sp.p
    r = sp.r
    s = sp.s    
    check_h = (f == MXV) || (cost(h,f+1,l,p,r,s) > MXS)
    check_f = (h == MXV) || (cost(h+1,f,l,p,r,s) > MXS)
    check_l = (l == MXV) || (cost(h,f,l+1,p,r,s) > MXS)
    check_prs = (p+r+s == 3*MXV) || (cost(h,f,l,p+1,r,s) > MXS)
    while !(check_h && check_f && check_l && check_prs)
        rand_i = rand(1:4)
        if rand_i == 1 && !check_h
            h = h+1
        end
        if rand_i == 2 && !check_f
            f = f+1
        end
        if rand_i == 3 && !check_l
            l = l+1
        end
        if rand_i == 4 && !check_prs
            rand_j = rand([1,1,1,1,1,2,2,2,2,3,3,3])
            if rand_j == 1 && p <= MXV
                p += 1
            end
            if rand_j == 2 && r <= MXV
                r += 1
            end
            if rand_j == 3 && s <= MXV
                s += 1
            end
        end            
        check_h = (f == MXV) || (cost(h,f+1,l,p,r,s) > MXS)
        check_f = (h == MXV) || (cost(h+1,f,l,p,r,s) > MXS)
        check_l = (l == MXV) || (cost(h,f,l+1,p,r,s) > MXS)
        check_prs = (p+r+s == 3*MXV) || (cost(h,f,l,p+1,r,s) > MXS)
    end
    return Spud(sp.name, h, f, l, p, r, s)
end



function pick_best_library(bs::Array{Spud})::Spud
    bestscore = -999
    bestf = Spud("",MNV,MNV,MNV,MNV,MNV,MNV)
    df = spuds_to_df(bs)
    hrange = vcat([MNV], df.h, df.h .+ 1, df.f, df.f .+ 1)
    frange = vcat([MNV], df.f, df.f .+ 1, df.h, df.h .+ 1)
    lrange = vcat([MNV], df.l, df.l .+ 1)
    prange = vcat([MNV], df.p, df.p .+ 1)
    rrange = vcat([MNV], df.r, df.r .+ 1)
    srange = vcat([MNV], df.s, df.s .+ 1)
    hrange = sort(unique(hrange))
    frange = sort(unique(frange))
    lrange = sort(unique(lrange))
    prange = sort(unique(prange))
    rrange = sort(unique(rrange))
    srange = sort(unique(srange))
    for h in hrange
        if (cost(h, MNV, MNV, MNV, MNV, MNV) < MXS)
            for f in frange
                if (cost(h, f, MNV, MNV, MNV, MNV) < MXS)
                    for l in lrange
                        if (cost(h, f, l, MNV, MNV, MNV) < MXS)
                            for p in prange
                                if (cost(h, f, l, p, MNV, MNV) < MXS)
                                    for r in rrange
                                        if (cost(h, f, l, p, r, MNV) < MXS)
                                            for s in srange
                                                if (cost(h,f,l,p,r,s) <= MXS)
                                                    ff = Spud("",h,f,l,p,r,s)
                                                    score = eval_battle_list(ff, bs)
                                                    if score > bestscore
                                                        bestscore = score
                                                        bestf = ff
                                                    end
                                                end
                                            end
                                        end
                                    end                        
                                end
                            end                        
                        end
                    end
                end
            end
        end
    end
    return upgrade_spud(bestf)
end



pick_best_library (generic function with 1 method)

In [8]:
function pick_ok_counter(bs::Array{Spud})::Spud
    bestscore = -999
    bestf = Spud("",MNV,MNV,MNV,MNV,MNV,MNV)
    df = spuds_to_df(bs)
    hrange = vcat([MNV], df.f .+ 1)
    frange = vcat([MNV], df.h .+ 1)
    lrange = vcat([MNV], df.l .+ 1)
    prange = vcat([MNV], df.p .+ 1)
    rrange = vcat([MNV], df.r .+ 1)
    srange = vcat([MNV], df.s .+ 1)
    hrange = sort(unique(hrange))
    frange = sort(unique(frange))
    lrange = sort(unique(lrange))
    prange = sort(unique(prange))
    rrange = sort(unique(rrange))
    srange = sort(unique(srange))
    for h in hrange
        if (cost(h, MNV, MNV, MNV, MNV, MNV) < MXS)
            for f in frange
                if (cost(h, f, MNV, MNV, MNV, MNV) < MXS)
                    for l in lrange
                        if (cost(h, f, l, MNV, MNV, MNV) < MXS)
                            for p in prange
                                if (cost(h, f, l, p, MNV, MNV) < MXS)
                                    for r in rrange
                                        if (cost(h, f, l, p, r, MNV) < MXS)
                                            for s in srange
                                                if (cost(h,f,l,p,r,s) <= MXS)
                                                    ff = Spud("",h,f,l,p,r,s)
                                                    score = eval_battle_list(ff, bs)
                                                    if score > bestscore
                                                        bestscore = score
                                                        bestf = ff
                                                    end
                                                end
                                            end
                                        end
                                    end                        
                                end
                            end                        
                        end
                    end
                end
            end
        end
    end
    return upgrade_spud(bestf)
end


pick_ok_counter (generic function with 1 method)

In [9]:
function pick_best_library2(bs::Array{Spud}, w::Vector{Float64})::Spud
    bestscore = -999.9
    bestf = Spud("",MNV,MNV,MNV,MNV,MNV,MNV)
    df = spuds_to_df(bs)
    hrange = vcat([MNV], df.h, df.h .+ 1, df.f, df.f .+ 1)
    frange = vcat([MNV], df.f, df.f .+ 1, df.h, df.h .+ 1)
    lrange = vcat([MNV], df.l, df.l .+ 1)
    prange = vcat([MNV], df.p, df.p .+ 1)
    rrange = vcat([MNV], df.r, df.r .+ 1)
    srange = vcat([MNV], df.s, df.s .+ 1)
    hrange = sort(unique(hrange))
    frange = sort(unique(frange))
    lrange = sort(unique(lrange))
    prange = sort(unique(prange))
    rrange = sort(unique(rrange))
    srange = sort(unique(srange))
    for h in hrange
        if (cost(h, MNV, MNV, MNV, MNV, MNV) < MXS)
            for f in frange
                if (cost(h, f, MNV, MNV, MNV, MNV) < MXS)
                    for l in lrange
                        if (cost(h, f, l, MNV, MNV, MNV) < MXS)
                            for p in prange
                                if (cost(h, f, l, p, MNV, MNV) < MXS)
                                    for r in rrange
                                        if (cost(h, f, l, p, r, MNV) < MXS)
                                            for s in srange
                                                if (cost(h,f,l,p,r,s) <= MXS)
                                                    ff = Spud("",h,f,l,p,r,s)
                                                    score = eval_battle_list2(ff, bs, w) + 0.001 * rand()
                                                    if score > bestscore
                                                        bestscore = score
                                                        bestf = ff
                                                    end
                                                end
                                            end
                                        end
                                    end                        
                                end
                            end                        
                        end
                    end
                end
            end
        end
    end
    return upgrade_spud(bestf)
end

pick_best_library2 (generic function with 1 method)

In [10]:
function subset_library(bs::Array{Spud}, n_hits::Int, w::Vector{Float64}, thres::Float64)::Array{Spud}
    bestf = Array{Spud}(undef, n_hits)
    spud_i = 0
    df = spuds_to_df(bs)
    hrange = vcat([MNV], df.h, df.h .+ 1, df.f, df.f .+ 1)
    frange = vcat([MNV], df.f, df.f .+ 1, df.h, df.h .+ 1)
    lrange = vcat([MNV], df.l, df.l .+ 1)
    prange = vcat([MNV], df.p, df.p .+ 1)
    rrange = vcat([MNV], df.r, df.r .+ 1)
    srange = vcat([MNV], df.s, df.s .+ 1)
    hrange = sort(unique(hrange))
    frange = sort(unique(frange))
    lrange = sort(unique(lrange))
    prange = sort(unique(prange))
    rrange = sort(unique(rrange))
    srange = sort(unique(srange))
    for h in hrange
        if (cost(h, MNV, MNV, MNV, MNV, MNV) < MXS)
            for f in frange
                if (cost(h, f, MNV, MNV, MNV, MNV) < MXS)
                    for l in lrange
                        if (cost(h, f, l, MNV, MNV, MNV) < MXS)
                            for p in prange
                                if (cost(h, f, l, p, MNV, MNV) < MXS)
                                    for r in rrange
                                        if (cost(h, f, l, p, r, MNV) < MXS)
                                            for s in srange
                                                if (cost(h,f,l,p,r,s) <= MXS)
                                                    ff = Spud("",h,f,l,p,r,s)
                                                    score = eval_battle_list2(ff, bs, w)
                                                    if score >= thres && spud_i < n_hits
                                                        spud_i += 1
                                                        bestf[spud_i] = upgrade_spud(ff)
                                                    end
                                                end
                                            end
                                        end
                                    end                        
                                end
                            end                        
                        end
                    end
                end
            end
        end
    end
    return unique(bestf[1:spud_i])
end

subset_library (generic function with 1 method)

In [11]:
# form initial library by subsampling indices
library = Array{Spud}(undef, 1000)
spud_i = 0
ss_prob = 0.01

0.01

In [12]:

hrange = MNV:MXV
frange = MNV:MXV
lrange = MNV:MXV
prange = MNV:MXV
rrange = MNV:MXV
srange = MNV:MXV

for h in hrange
    if (cost(h, MXV, MXV, MXV, MXV, MXV) >= MXS) && (cost(h, MNV, MNV, MNV, MNV, MNV) < MXS)
        for f in frange
            if (cost(h, f, MXV, MXV, MXV, MXV) >= MXS) && (cost(h, f, MNV, MNV, MNV, MNV) < MXS)
                for l in lrange
                    if (cost(h, f, l, MXV, MXV, MXV) >= MXS) && (cost(h, f, l, MNV, MNV, MNV) < MXS)
                        for p in prange
                            if (cost(h, f, l, p, MXV, MXV) >= MXS) && (cost(h, f, l, p, MNV, MNV) < MXS)
                                for r in rrange
                                    if (cost(h, f, l, p, r, MXV) >= MXS) && (cost(h, f, l, p, r, MNV) < MXS)
                                        for s in srange
                                            if rand() < ss_prob && (cost(h,f,l,p,r,s) <= MXS)
                                                check_f = (h == MXV) || (cost(h+1,f,l,p,r,s) > MXS)
                                                check_h = (f == MXV) || (cost(h,f+1,l,p,r,s) > MXS)
                                                check_l = (l == MXV) || (cost(h,f,l+1,p,r,s) > MXS)
                                                check_prs = (p+r+s == 3*MXV) || (cost(h,f,l,p+1,r,s) > MXS)
                                                if check_h && check_f && check_l && check_prs
                                                    spud_i += 1
                                                    #randname = rand_rename(Spud(" ",h,f,l,p,r,s)).name
                                                    #name = string("#", @sprintf("%i", spud_i), ". ", randname)
                                                    name = ""
                                                    library[spud_i] = Spud(name,h,f,l,p,r,s)
                                                end
                                            end
                                        end
                                    end
                                end                        
                            end
                        end                        
                    end
                end
            end
        end
    end
end


In [13]:
library = unique(library[1:spud_i])
n_spuds = length(library)

713

In [14]:
# for i in 1:n_spuds
#     ff = library[i]
#     randname = rand_rename(ff).name
#     name = string("#", @sprintf("%i", i), ". ", randname)
#     library[i] = Spud(name,ff.h,ff.f,ff.l,ff.p,ff.r,ff.s)
# end
# id_no = Dict(library[i] => i for i in 1:n_spuds)

In [15]:
# Check that there are no ties in spudland
for iter in 1:10000
    i = rand(1:n_spuds)
    j = rand(1:n_spuds)
    if i != j && eval_battle(library[i], library[j]) == 0
        println(library[i])
        println(library[j])
        println()
    end
end

## Compute Nash env

In [16]:
function ffp(nash_env, nits)
    n_nash = length(nash_env)
    i_lose = Array{Int}(undef, (n_nash, n_nash))
    n_lose = Array{Int}(undef, n_nash)
    for i in 1:n_nash
        n_lose[i] = 0
        ff = nash_env[i]
        for j in 1:n_nash
            if eval_battle(ff, nash_env[j]) ==-1
                n_lose[i] += 1
                i_lose[i, n_lose[i]] = j
            end
        end
    end
    counts = [0 for i in 1:n_nash]
    wins = [0 for i in 1:n_nash]
    for i in 1:n_nash
        counts[i] += 1
        for j in 1:n_lose[i]
            i_w = i_lose[i, j]
            wins[i_w]+= 1
        end
    end
    for iter in 1:nits
        ind_winners = findall(wins .== maximum(wins))
        i = rand(ind_winners)
        counts[i] += 1
        for j in 1:n_lose[i]
            i_w = i_lose[i, j]
            wins[i_w]+= 1
        end
    end
    return counts
end


ffp (generic function with 1 method)

In [17]:
nash_env = [rand_rename(s) for s in library]
spuds_to_df(nash_env)

Row,name,h,f,l,p,r,s
,String,Int64,Int64,Int64,Int64,Int64,Int64
1,Unfair Batata vada the Coast guard,1,2,2,20,10,18
2,Western Okroshka the Software developer,1,2,6,12,16,18
3,Sure Hash browns the Dictator,1,2,10,17,14,13
4,Proud French fries the Technical engineer,1,2,12,16,12,15
5,Confident Brændende kærlighed the System designer,1,2,12,16,18,9
6,Intelligent Potato babka the Medic,1,2,16,6,20,15
7,Efficient Pitepalt the Signalman,1,2,16,7,19,15
8,Professional Rewena bread the Technical engineer,1,2,16,10,13,18
9,Wild Suan la tu dou si the Etymologist,1,2,16,17,10,14


In [18]:
counts = ffp(nash_env, 1000);

In [19]:
nash_env = nash_env[counts .> 10]
spuds_to_df(nash_env)

Row,name,h,f,l,p,r,s
,String,Int64,Int64,Int64,Int64,Int64,Int64
1,Powerful Okroshka the Queen mother,3,5,11,2,5,3
2,Open Chapalele the Gondolier,3,9,3,2,3,2
3,Strange Pommes Anna the Gondolier,4,1,18,2,10,12
4,Strange French fries the Gondolier,4,3,14,6,2,2
5,Upper Älplermagronen the Telephone operator,4,7,5,2,1,4
6,Negative Suan la tu dou si the Demolitionist,5,1,15,15,3,2
7,Independent Okroshka the Demolitionist,5,2,14,6,1,3
8,Serious Pommes Anna the Demolitionist,5,3,13,3,2,1
9,Dangerous Boulangère potatoes the Laundress (also Lavendar),5,6,2,1,4,5


In [20]:
counts = ffp(nash_env, 10000)

22-element Vector{Int64}:
  883
 1264
  159
  696
    1
  569
    1
    1
  380
  407
    1
  820
  285
  348
    1
    1
 1168
    3
    1
 1390
 1072
  571

In [21]:
nash_env = nash_env[counts .> 10]
spuds_to_df(nash_env)

Row,name,h,f,l,p,r,s
,String,Int64,Int64,Int64,Int64,Int64,Int64
1,Powerful Okroshka the Queen mother,3,5,11,2,5,3
2,Open Chapalele the Gondolier,3,9,3,2,3,2
3,Strange Pommes Anna the Gondolier,4,1,18,2,10,12
4,Strange French fries the Gondolier,4,3,14,6,2,2
5,Negative Suan la tu dou si the Demolitionist,5,1,15,15,3,2
6,Dangerous Boulangère potatoes the Laundress (also Lavendar),5,6,2,1,4,5
7,Expensive Clapshot the Party-leader,5,6,6,1,4,1
8,Fresh Pommes Anna the Impersonator,6,1,13,3,2,11
9,Quiet Boulangère potatoes the Juggler,6,5,4,2,3,4


### Loop this

In [22]:
counts = ffp(nash_env, 100)

14-element Vector{Int64}:
 10
 15
  2
  7
  6
  4
  5
  9
  4
  3
 13
 17
 12
  7

In [23]:
#counts = ffp(nash_env, 10000)

In [24]:
spuds_to_df(nash_env)

Row,name,h,f,l,p,r,s
,String,Int64,Int64,Int64,Int64,Int64,Int64
1,Powerful Okroshka the Queen mother,3,5,11,2,5,3
2,Open Chapalele the Gondolier,3,9,3,2,3,2
3,Strange Pommes Anna the Gondolier,4,1,18,2,10,12
4,Strange French fries the Gondolier,4,3,14,6,2,2
5,Negative Suan la tu dou si the Demolitionist,5,1,15,15,3,2
6,Dangerous Boulangère potatoes the Laundress (also Lavendar),5,6,2,1,4,5
7,Expensive Clapshot the Party-leader,5,6,6,1,4,1
8,Fresh Pommes Anna the Impersonator,6,1,13,3,2,11
9,Quiet Boulangère potatoes the Juggler,6,5,4,2,3,4


In [25]:
# # don't run this every iteration!!
# nash_env = nash_env[counts .> 10]
# counts = ffp(nash_env, 10000)

In [26]:
ff = rand_rename(pick_best_library2(nash_env, counts./sum(counts)))

Spud("Open Boulangère potatoes the Quilter", 1, 9, 1, 3, 3, 4)

In [27]:
ev = eval_battle_list2(ff, nash_env, counts./sum(counts))

0.7894736842105263

In [28]:
bestf = subset_library(nash_env, 500, counts./sum(counts), ev-0.05)

2-element Vector{Spud}:
 Spud("", 1, 9, 1, 3, 3, 4)
 Spud("", 1, 9, 1, 3, 4, 3)

In [29]:
cc = ffp(bestf, 10000)
println(maximum(cc))
ff2 = rand_rename(bestf[cc .== maximum(cc)][1])

10001


Spud("Warm Cacasse à cul nu the Telephone operator", 1, 9, 1, 3, 4, 3)

In [30]:
eval_battle_list2(ff2, nash_env, counts./sum(counts))

0.7543859649122807

In [31]:
append!(nash_env, [ff2])
spuds_to_df(nash_env)

Row,name,h,f,l,p,r,s
,String,Int64,Int64,Int64,Int64,Int64,Int64
1,Powerful Okroshka the Queen mother,3,5,11,2,5,3
2,Open Chapalele the Gondolier,3,9,3,2,3,2
3,Strange Pommes Anna the Gondolier,4,1,18,2,10,12
4,Strange French fries the Gondolier,4,3,14,6,2,2
5,Negative Suan la tu dou si the Demolitionist,5,1,15,15,3,2
6,Dangerous Boulangère potatoes the Laundress (also Lavendar),5,6,2,1,4,5
7,Expensive Clapshot the Party-leader,5,6,6,1,4,1
8,Fresh Pommes Anna the Impersonator,6,1,13,3,2,11
9,Quiet Boulangère potatoes the Juggler,6,5,4,2,3,4


### loop

In [32]:
counts = ffp(nash_env, 100000)
println(maximum([eval_battle_list2(ff, nash_env, counts./sum(counts)) for ff in nash_env]))
ff = rand_rename(pick_best_library2(nash_env, counts./sum(counts)))
ev = eval_battle_list2(ff, nash_env, counts./sum(counts))
println(ev)

0.0882367644853272
0.9117332400139979


In [33]:
bestf = subset_library(nash_env, 900, counts./sum(counts), ev-0.05)

2-element Vector{Spud}:
 Spud("", 2, 7, 11, 3, 5, 1)
 Spud("", 2, 7, 11, 4, 1, 4)

In [34]:
println(length(bestf))
cc = ffp(bestf, 10000)
println(maximum(cc))

2
10001


In [35]:
ff2 = rand_rename(bestf[cc .== maximum(cc)][1])

Spud("Creative Tartiflette the Gondolier", 2, 7, 11, 4, 1, 4)

In [36]:
println(eval_battle_list2(ff2, nash_env, counts./sum(counts)))

0.911713243013548


In [37]:
append!(nash_env, [ff2])

16-element Vector{Spud}:
 Spud("Powerful Okroshka the Queen mother", 3, 5, 11, 2, 5, 3)
 Spud("Open Chapalele the Gondolier", 3, 9, 3, 2, 3, 2)
 Spud("Strange Pommes Anna the Gondolier", 4, 1, 18, 2, 10, 12)
 Spud("Strange French fries the Gondolier", 4, 3, 14, 6, 2, 2)
 Spud("Negative Suan la tu dou si the Demolitionist", 5, 1, 15, 15, 3, 2)
 Spud("Dangerous Boulangère potatoes the Laundress (also Lavendar)", 5, 6, 2, 1, 4, 5)
 Spud("Expensive Clapshot the Party-leader", 5, 6, 6, 1, 4, 1)
 Spud("Fresh Pommes Anna the Impersonator", 6, 1, 13, 3, 2, 11)
 Spud("Quiet Boulangère potatoes the Juggler", 6, 5, 4, 2, 3, 4)
 Spud("Clean Chips and dip the Housekeeper", 6, 5, 6, 1, 2, 3)
 Spud("Soft Chapalele the Organizer", 7, 7, 1, 1, 3, 2)
 Spud("Tough Gamjajeon the Party-leader", 8, 3, 5, 3, 2, 7)
 Spud("Nervous Kouign patatez the Dictator", 9, 1, 9, 6, 1, 3)
 Spud("Boring Pitepalt the Shoeshiner", 9, 4, 4, 3, 2, 2)
 Spud("Warm Cacasse à cul nu the Telephone operator", 1, 9, 1, 3, 4, 3)
 Spu

In [52]:
for ii in 1:80
    println()
    println(ii)
    counts = ffp(nash_env, 100000)
    println(maximum([eval_battle_list2(ff, nash_env, counts./sum(counts)) for ff in nash_env]))
    ff = rand_rename(pick_best_library2(nash_env, counts./sum(counts)))
    ev = eval_battle_list2(ff, nash_env, counts./sum(counts))
    println(ev)
    bestf = subset_library(nash_env, 900, counts./sum(counts), ev-0.05)
    println(length(bestf))
    cc = ffp(bestf, 10000)
    println(maximum(cc))
    ff2 = rand_rename(bestf[cc .== maximum(cc)][1])
    println(eval_battle_list2(ff2, nash_env, counts./sum(counts)))
    append!(nash_env, [ff2])
end


1
0.032091707726595836
0.15758017929780022
14
2002
0.10792848076596337

2
0.02491554898159068
0.14813408223230526
105
2242
0.1108756920985828

3
0.04141556481675811
0.14422490730469023
66
1922
0.10488811601155315

4
0.034949030581651004
0.16618029182490504
78
2960
0.13359984009594242

5
0.03105105885409902
0.16832732033459596
94
1910
0.13522751121815696

6
0.0375167396214347
0.18452559413163835
4
10001
0.13549599248465952

7
0.039245275476449855
0.14825659834304386
61
1454
0.11825549903560756

8
0.03530740326191239
0.16803245922609533
12
3338
0.12220179085385355

9
0.03193923949432869
0.18592914605506422
19
3341
0.17089891570479188

10
0.025792976635420593
0.14074710690943984
58
1991
0.09977414906161935

11
0.028041212387700225
0.16300078947105434
9
10001
0.1295631926609172

12
0.02703161849942039
0.15769276891713638
27
3758
0.11038493824199548

13
0.020236037134377265
0.18732074868340834
17
3757
0.1465488812719223

14
0.023283701409013705
0.20989307484760658
1
10001
0.209893074847606

In [53]:
#spuds_to_df(nash_env)

In [54]:
counts = ffp(nash_env, 100000)
ff = pick_best_library2(nash_env, counts./sum(counts))
println(ff)
eval_battle_list2(ff, nash_env, counts./sum(counts))

Spud("", 9, 2, 7, 2, 2, 5)


0.10906058699581568

In [55]:
#CSV.write("spuds_nash_stage12.csv", spuds_to_df(nash_env))

In [56]:
#counts = ffp(nash_env, 10000)
nash_env = nash_env[counts .> 10]
counts = ffp(nash_env, 100000);

In [57]:
#counts = ffp(nash_env, 100000)
ff = pick_best_library2(nash_env, counts./sum(counts))
println(ff)
eval_battle_list2(ff, nash_env, counts./sum(counts))

Spud("", 9, 2, 7, 2, 2, 5)


0.10913959157575032

## beware

In [58]:
CSV.write("spuds_nash_stage03.csv", spuds_to_df(nash_env))

"spuds_nash_stage03.csv"

### refine weights

In [149]:
eps = 0.01/length(library)
nits = 40000
sub_iter = 2000
w = counts./sum(counts)

19-element Vector{Float64}:
 0.03144026349935123
 0.051801577003692985
 0.03992414412616029
 0.040822437368998904
 0.03992414412616029
 0.07156402834614232
 0.00948198422996307
 0.011478191436271086
 0.04820840403233856
 0.10959177562631
 0.04301826529593772
 0.05928735402734804
 0.05589380177662441
 0.006687294141131849
 0.005090328376085438
 0.03902585088332169
 0.09551851482183851
 0.051801577003692985
 0.1894400638786306

In [150]:
n_nash = length(nash_env)
i_lose = Array{Int}(undef, (n_nash, n_nash))
n_lose = Array{Int}(undef, n_nash)
for i in 1:n_nash
    n_lose[i] = 0
    ff = nash_env[i]
    for j in 1:n_nash
        if eval_battle(ff, nash_env[j]) ==-1
            n_lose[i] += 1
            i_lose[i, n_lose[i]] = j
        end
    end
end

In [152]:
scores = [0.0 for i in 1:n_nash]
for i in 1:n_nash
    for j in 1:n_lose[i]
        i_w = i_lose[i, j]
        scores[i_w]+= w[i]
    end
end

In [153]:
previous_score = maximum(scores)
for iter in 1:nits
    ind_winners = findall(scores .== maximum(scores))
    i = rand(ind_winners)
    w[i] += eps
    for j in 1:n_lose[i]
        i_w = i_lose[i, j]
        scores[i_w]+= eps
    end
    if mod1(iter, sub_iter) == 1
        scores = scores./sum(w)
        w = w./sum(w)
        if maximum(scores) > previous_score
            eps = eps/2
            println(eps)
            println()
            println()
        end
        previous_score = maximum(scores)
        mxv, mxi = findmax(scores)
        print(nash_env[mxi])
        print(" w:")
        print(w[mxi])
        print(" s:")
        print(scores[mxi])
        println()
    end
end


2.4630541871921184e-5


Spud("Hungry French Fries", 2, 16, 8, 2, 11, 9) w:0.04820602935108973 s:0.45715819122033735
Spud("Potato Strips w/ onion", 9, 21, 2, 4, 2, 3) w:0.18955135521145414 s:0.45700869861606747
1.2315270935960592e-5


Spud("Aquatic Mashed Potato", 14, 6, 13, 4, 11, 7) w:0.10971268975376745 s:0.4570093923044257
Spud("Crazy Chips", 6, 9, 26, 8, 3, 10) w:0.05587069246628821 s:0.45697416637016347
Spud("Friendly Potato Bread", 4, 12, 19, 4, 9, 10) w:0.05963552991975216 s:0.4569652237420019
6.157635467980296e-6


Spud("Potato Soup w/ ketchup", 21, 2, 15, 7, 6, 8) w:0.04319174825362203 s:0.4569657070545698
3.078817733990148e-6


Spud("Crazy Potato Soup", 8, 4, 24, 18, 4, 22) w:0.05181948311653167 s:0.45696838203738316
Spud("Hungry French Fries", 2, 16, 8, 2, 11, 9) w:0.048096443986402025 s:0.4569578797912595
1.539408866995074e-6


Spud("Aquatic Mashed Potato", 14, 6, 13, 4, 11, 7) w:0.10973879429331644 s:0.45695842146670224
Spud("Potato Soup w/ onion", 19, 13, 3, 3, 2, 2) w:0.

In [154]:
maximum([eval_battle_list2(ff, nash_env, w) for ff in nash_env])

0.10350172839487909

In [148]:
ff = pick_best_library2(nash_env, w)
println(ff)
eval_battle_list2(ff, nash_env, w)

Spud("", 17, 9, 9, 6, 2, 2)


0.5977873078679817

## Old code

In [15]:
function nashprox2(size, start_size, ns)
    nash_env = Array{Spud}(undef, size)
    nash_env[1:start_size] = random_team(() -> rand(library), start_size)
    for j in (start_size+1):size
        if j < ns
            ff = pick_ok_counter(nash_env[1:(j-1)])
            nash_env[j] = ff
        else
            ff = pick_ok_counter(randsubseq(nash_env[1:(j-1)], ns/j))
            nash_env[j] = ff                
        end
    end
    return nash_env
end

function nashprox(size, start_size, ns)
    nash_env = Array{Spud}(undef, size)
    nash_env[1:start_size] = random_team(() -> rand(library), start_size)
    for j in (start_size+1):size
        if j < ns
            ff = pick_best_library(nash_env[1:(j-1)])
            nash_env[j] = ff
        else
            ff = pick_best_library(randsubseq(nash_env[1:(j-1)], ns/j))
            nash_env[j] = ff                
        end
    end
    return nash_env
end

nashprox (generic function with 1 method)

In [150]:
@time nash_env = nashprox(4000, 10, 100)
nash_env = unique(nash_env)
#inds_nash = [id_no[ff] for ff in nash_env];
#length(inds_nash)
n_nash = length(nash_env)

201.611800 seconds (15.12 k allocations: 197.526 MiB, 0.01% gc time)


1540

In [151]:
# Verify 3-explotability
for i in 1:1000
    #inds = sort([rand(inds_nash) for i in 1:3])
    #rt = library[inds]
    rt = random_team(() -> rand(nash_env), 3)
    ff = pick_best_library(rt)
    ev = eval_battle_list(ff, rt)
    if ev < 3
        println(rt)
    end
end


In [152]:
# Verify 4-explotability
for i in 1:1000
    #inds = sort([rand(inds_nash) for i in 1:3])
    #rt = library[inds]
    rt = random_team(() -> rand(nash_env), 4)
    ff = pick_best_library(rt)
    ev = eval_battle_list(ff, rt)
    if ev < 4
        println(rt)
        println(ev)
    end
end


Spud[Spud("", 6, 3, 9, 5, 3, 1), Spud("", 7, 9, 1, 3, 1, 2), Spud("", 15, 5, 2, 1, 1, 2), Spud("", 12, 1, 6, 1, 8, 6)]
2
Spud[Spud("", 11, 6, 2, 3, 1, 1), Spud("", 2, 10, 3, 4, 3, 2), Spud("", 10, 3, 5, 1, 4, 4), Spud("", 5, 1, 14, 5, 4, 3)]
2


In [103]:
#inds = sort([rand(1:n_spuds) for i in 1:5])
#rt = library[inds]

In [104]:
# rt = library[[1403,1902,2283,2397]]
# rt

In [105]:
# ff = pick_best(library, rt)
# ind = [i for i in 1:n_spuds if library[i] == ff][1]
# print(ff)
# print(" ")
# eval_battle_list(ff, rt)

## Fictitious Play to compute Nash distribution

In [135]:
@time nash_env = nashprox(4000, 10, 100)
nash_env = unique(nash_env)
#inds_nash = [id_no[ff] for ff in nash_env];
#length(inds_nash)
n_nash = length(nash_env)

In [153]:
i_lose = Array{Int}(undef, (n_nash, n_nash))
n_lose = Array{Int}(undef, n_nash)
for i in 1:n_nash
    n_lose[i] = 0
    ff = nash_env[i]
    for j in 1:n_nash
        if eval_battle(ff, nash_env[j]) ==-1
            n_lose[i] += 1
            i_lose[i, n_lose[i]] = j
        end
    end
end

In [154]:
# initialize FP with 1 of each
counts = [0 for i in 1:n_nash]
wins = [0 for i in 1:n_nash]
for i in 1:n_nash
    counts[i] += 1
    for j in 1:n_lose[i]
        i_w = i_lose[i, j]
        wins[i_w]+= 1
    end
end

In [155]:
nits = 500000
for iter in 1:nits
    ind_winners = findall(wins .== maximum(wins))
    for i in ind_winners
        counts[i] += 1
        for j in 1:n_lose[i]
            i_w = i_lose[i, j]
            wins[i_w]+= 1
        end
    end
    if mod1(iter, 10000) == 1
        mxv, mxi = findmax(wins)
        print(nash_env[mxi])
        print(" n:")
        print(counts[mxi])
        print(" w:")
        print(wins[mxi])
        println()
    end
end

w = counts./sum(counts)
println(maximum([eval_battle_list2(ff, nash_env, w) for ff in nash_env]))
sum(counts)

Spud("", 9, 10, 1, 1, 1, 1) n:2 w:1035
Spud("", 13, 6, 2, 1, 1, 1) n:235 w:10618
Spud("", 9, 7, 4, 1, 1, 1) n:90 w:20654
Spud("", 15, 1, 6, 3, 1, 2) n:565 w:30681
Spud("", 1, 7, 1, 1, 6, 7) n:266 w:40725
Spud("", 5, 11, 1, 2, 2, 2) n:356 w:50746
Spud("", 2, 9, 2, 5, 4, 1) n:272 w:60757
Spud("", 1, 7, 1, 8, 1, 5) n:632 w:70831
Spud("", 2, 14, 1, 3, 1, 3) n:596 w:80868
Spud("", 3, 6, 10, 5, 1, 3) n:1513 w:90906
Spud("", 5, 4, 9, 2, 5, 2) n:288 w:100925
Spud("", 6, 11, 1, 3, 1, 1) n:1317 w:110944
Spud("", 5, 4, 8, 6, 3, 1) n:318 w:120973
Spud("", 6, 13, 2, 1, 1, 1) n:3551 w:131027
Spud("", 7, 10, 1, 1, 2, 2) n:838 w:141075
Spud("", 10, 1, 8, 3, 2, 5) n:750 w:151124
Spud("", 2, 4, 14, 8, 1, 4) n:1200 w:161191
Spud("", 4, 2, 14, 4, 3, 5) n:1234 w:171225
Spud("", 9, 1, 10, 1, 1, 2) n:7296 w:181302
Spud("", 1, 13, 4, 4, 1, 3) n:1101 w:191329
Spud("", 9, 3, 5, 1, 5, 5) n:1149 w:201353
Spud("", 13, 5, 2, 1, 3, 2) n:1581 w:211379
Spud("", 13, 5, 2, 3, 1, 2) n:3510 w:221392
Spud("", 12, 1, 7, 3, 

1010287

In [156]:
ff = pick_best_library2(nash_env, w)
println(ff)
eval_battle_list2(ff, nash_env, w)

Spud("", 16, 1, 5, 5, 5, 1)


0.03152866462698188

In [110]:
inds_sel = sortperm(-w)[1:500]
tab = spuds_to_df(nash_env[inds_sel])
tab[!, :w] = w[inds_sel]
tab

Row,name,h,f,l,p,r,s,w
,String,Int64,Int64,Int64,Int64,Int64,Int64,Float64
1,#6705. Potato Strips w/ scallion,8,10,1,1,2,1,0.0341574
2,#2813. Helen's Chips,6,1,14,2,1,2,0.0266422
3,#9066. Potato Soup w/ scallion,14,5,2,1,2,2,0.0256009
4,#6499. Potato Soup w/ ketchup,8,1,10,5,2,2,0.0237062
5,#3007. Helen's Mashed Potato,11,1,8,3,1,2,0.0213634
6,#7179. Potato Soup w/ onion,11,4,3,5,2,2,0.0200976
7,#7309. Potato Soup w/ scallion,14,2,5,3,2,2,0.0185596
8,#3197. Potato Soup w/ scallion,14,3,4,3,1,2,0.0179148
9,#307. Baked Potato w/ onion,3,15,3,2,2,1,0.0172551


In [111]:
# Verify explotability
for i in 1:10000
    inds = sort([rand(inds_sel[1:120]) for i in 1:3])
    rt = nash_env[inds]
    ff = pick_best(library, rt)
    ev = eval_battle_list(ff, rt)
    if ev < 3
        println(spuds_to_df(rt))
        println(w[inds])
        println(ev)
    end
end

3×7 DataFrame
 Row │ name                               h      f      l      p      r      s     
     │ String                             Int64  Int64  Int64  Int64  Int64  Int64 
─────┼─────────────────────────────────────────────────────────────────────────────
   1 │ #2248. Sneaky Mashed Potato           11      3      4      1      5      4
   2 │ #5770. Helen's Potato Bread w/ k…      6      2     10      7      3      1
   3 │ #6951. Potato Soup w/ scallion        10      7      1      3      2      1
[0.007680178680897021, 0.00268458509647914, 0.0082961826630647]
1
3×7 DataFrame
 Row │ name                           h      f      l      p      r      s     
     │ String                         Int64  Int64  Int64  Int64  Int64  Int64 
─────┼─────────────────────────────────────────────────────────────────────────
   1 │ #1764. Friendly Tater Tots         3      3     12      1      7      6
   2 │ #3097. Charles' Potato Soup       12      1      7      4      1      3
   3 │ 

3×7 DataFrame
 Row │ name                               h      f      l      p      r      s     
     │ String                             Int64  Int64  Int64  Int64  Int64  Int64 
─────┼─────────────────────────────────────────────────────────────────────────────
   1 │ #2240. Criminal Mashed Potato         10      3      5      5      3      1
   2 │ #10984. Helen's Home Fries             7      2      9      5      3      2
   3 │ #14881. Angry Mashed Potato w/ s…      6      8      1      1      4      3
[0.011917689945324678, 0.007105904000747153, 0.006316027926838598]
1
3×7 DataFrame
 Row │ name                              h      f      l      p      r      s     
     │ String                            Int64  Int64  Int64  Int64  Int64  Int64 
─────┼────────────────────────────────────────────────────────────────────────────
   1 │ #1764. Friendly Tater Tots            3      3     12      1      7      6
   2 │ #3093. Sneaky Mashed Potato          12      1      7      2    

In [60]:
rt = nash_env[sort([rand(1:n_nash) for i in 1:4])]

4-element Vector{Spud}:
 Spud("#7981. Mashed Potato w/ ketchup", 3, 1, 11, 2, 4, 3)
 Spud("#8650. Charles' Potato Bread", 3, 2, 10, 2, 2, 3)
 Spud("#10705. Devious Baked Potato w/ ketchup", 4, 3, 4, 5, 2, 2)
 Spud("#14443. Sneaky Potato Soup", 8, 2, 2, 1, 5, 6)

In [61]:
ff = pick_best_rdmly(nash_env[inds_sel], rt, 99)
ev = eval_battle_list(ff, rt)
println(ff)
println(ev)
eval_battle_list2(ff, nash_env, w)

Spud("#12249. Enlightened Mashed Potato", 5, 6, 4, 1, 1, 1)
4


-0.05855215021150282

In [66]:
ff = pick_best_rdmly(library, rt, 99)
ev = eval_battle_list(ff, rt)
println(ff)
println(ev)
eval_battle_list2(ff, nash_env, w)

Spud("#9053. Aquatic French Fries", 3, 4, 4, 1, 4, 3)
4


-0.1656790165129023

In [35]:
ff = rand(library)
println(ff)
eval_battle_list2(ff, nash_env, w)

Spud("#14744. Battle Potato Soup", 9, 1, 1, 13, 6, 7)


-0.6779939500016593

In [23]:
#cost(4,4,9,1,4,2)

In [24]:
#rt = library[[15918,21020,26191]]
#eval_battle(Spud("",4,4,9,1,4,1), rt[1])

In [25]:
#sum(sort(-w)[1:450])

In [26]:
#sum(w)

In [27]:
# # FP is slow to converge in the end
# eps = 0.01/length(library)
# nits = 20000
# scores = wins./sum(counts)
# w = counts./sum(counts)
# 0

In [28]:
# previous_score = maximum(scores)
# for iter in 1:nits
#     ind_winners = findall(wins .== maximum(wins))
#     i = rand(ind_winners)
#     w[i] += eps
#     for j in 1:n_lose[i]
#         i_w = i_lose[i, j]
#         scores[i_w]+= eps
#     end
#     if mod1(iter, 2000) == 1
#         scores = scores./sum(w)
#         w = w./sum(w)
#         if maximum(scores) > previous_score
#             eps = eps/2
#             println(eps)
#             println()
#             println()
#         end
#         previous_score = maximum(scores)
#         mxv, mxi = findmax(wins)
#         print(library[mxi])
#         print(" n:")
#         print(w[mxi])
#         print(" w:")
#         print(scores[mxi])
#         println()
#         mxv, mxi = findmax(counts)
#         print(library[mxi])
#         print(" n:")
#         print(w[mxi])
#         print(" w:")
#         print(scores[mxi])
#         println()
#         println(maximum([eval_battle_list2(ff, library, w) for ff in library]))
#         println()
#     end
# end
